### Imports

In [10]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


from utils.dataset.dataset import create_fiftyone_dataset
from utils.dataset.dataset import FiftyOneTorchDataset
from utils.models.detr_demo import DETRdemo

import wandb

import os
import sys

sys.path.append("./vision/references/detection/")

from vision.references.detection.engine import train_one_epoch, evaluate
import vision.references.detection.utils as utils
import vision.references.detection.transforms as T


### Creating Datasets

In [2]:
wanted_classes = "signals"

wanted_model = "RCNN"
#wanted_model = "DETR"

In [3]:
wandb.init(project = 'faster_rcnn', name =  wanted_model + '/' + wanted_classes)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: stanislasdozias. Use `wandb login --relogin` to force relogin


In [4]:
def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [5]:
fo_train_set = create_fiftyone_dataset(wanted_classes, "train", drop_existing_dataset=False)#.take(20, seed=51)
fo_test_set = create_fiftyone_dataset(wanted_classes, "validation", drop_existing_dataset=False)#.take(10, seed=51)

fo_train_set.compute_metadata()
fo_test_set.compute_metadata()

train_set = FiftyOneTorchDataset(fo_train_set, get_transform(train=True))
test_set = FiftyOneTorchDataset(fo_test_set, get_transform(train=False))
num_classes = len(train_set.classes)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=True, num_workers=0, collate_fn=utils.collate_fn)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=0, collate_fn=utils.collate_fn)

Found annotations at 'C:\Users\stani\fiftyone\coco-2017\raw\instances_train2017.json'
Sufficient images already downloaded
Existing download of split 'train' is sufficient
Loading 'coco-2017' split 'train'
 100% |█████████████| 12880/12880 [38.1s elapsed, 0s remaining, 351.6 samples/s]      
Dataset 'train_signals' created
Found annotations at 'C:\Users\stani\fiftyone\coco-2017\raw\instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |█████████████████| 560/560 [1.9s elapsed, 0s remaining, 302.8 samples/s]      
Dataset 'validation_signals' created


In [7]:
if wanted_model == "RCNN" :
    # load a model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

if wanted_model == "DETR" :
    model = DETRdemo(num_classes=num_classes)

c:\Users\stani\anaconda3\envs\coco\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\Users\stani\anaconda3\envs\coco\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
if not os.path.isdir("results/"+wanted_model+"/"):
    os.makedirs("results/"+wanted_model+"/")

In [ ]:
num_epochs = 10
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device %s" % device)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

for epoch in range(num_epochs):
    to_log = {}

    # train for one epoch, printing every 10 iterations
    train_metrics = train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=1000)

    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    test_metric = evaluate(model, test_loader, device=device)

    test_coco_eval = test_metric.coco_eval["bbox"]
    precision = test_coco_eval.stats[0]
    recall = test_coco_eval.stats[8]
    f1_score = 2 * precision * recall / (precision + recall)

    to_log["main/train_loss"] = train_metrics.loss.value
    to_log["main/precision"] = precision
    to_log["main/recall"] = recall
    to_log["main/f1_score"] = f1_score
    wandb.log(to_log)
    torch.save(model.state_dict(), "results/"+wanted_model+"/"+wanted_classes+".pt")

Using device cuda
Epoch: [0]  [    0/12880]  eta: 13:27:03  lr: 0.000010  loss: 2.3270 (2.3270)  loss_classifier: 2.1300 (2.1300)  loss_box_reg: 0.1945 (0.1945)  loss_objectness: 0.0020 (0.0020)  loss_rpn_box_reg: 0.0004 (0.0004)  time: 3.7596  data: 0.0383  max mem: 1204
Epoch: [0]  [ 1000/12880]  eta: 1:21:51  lr: 0.005000  loss: 0.2189 (0.3612)  loss_classifier: 0.1047 (0.1469)  loss_box_reg: 0.0583 (0.1267)  loss_objectness: 0.0456 (0.0533)  loss_rpn_box_reg: 0.0069 (0.0343)  time: 0.4178  data: 0.0170  max mem: 1662
Epoch: [0]  [ 2000/12880]  eta: 1:19:09  lr: 0.005000  loss: 0.2050 (0.3351)  loss_classifier: 0.0501 (0.1205)  loss_box_reg: 0.0353 (0.1062)  loss_objectness: 0.0598 (0.0662)  loss_rpn_box_reg: 0.0149 (0.0422)  time: 0.4141  data: 0.0176  max mem: 1662
Epoch: [0]  [ 3000/12880]  eta: 1:13:34  lr: 0.005000  loss: 0.1999 (0.3222)  loss_classifier: 0.0795 (0.1122)  loss_box_reg: 0.0583 (0.0994)  loss_objectness: 0.0392 (0.0679)  loss_rpn_box_reg: 0.0085 (0.0427)  time: 0

main/f1_score,▄▁▃▇▇▇████
main/precision,▄▁▃▇▇▇████
main/recall,▄▁▃▇▇▇████
main/train_loss,▅▂▂▄▁▃▄█▁▄
main/f1_score,0.49655
main/precision,0.43847
main/recall,0.57237
main/train_loss,0.21848
